Importing the necessary libraries

In [8]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

trying to get the shape of data and removing unncessary columns

In [9]:
df = pd.read_csv("books.csv")
df_book_tags = pd.read_csv("book_tags.csv")
df_tags = pd.read_csv("tags.csv")
# 
df = df.drop([
    'work_id', 'isbn', 'isbn13', 'language_code', 'image_url', 'small_image_url', 'best_book_id', 'books_count','ratings_1', 'ratings_2', 'ratings_3', 'ratings_4', 'ratings_5'
],axis = 1)
df.dtypes



book_id                        int64
goodreads_book_id              int64
authors                       object
original_publication_year    float64
original_title                object
title                         object
average_rating               float64
ratings_count                  int64
work_ratings_count             int64
work_text_reviews_count        int64
dtype: object

In [10]:
df_tags_named = pd.merge(
    df_book_tags,
    df_tags,
    on='tag_id',
    how='left' # Keep all tag records, even if a tag_name is missing (rare)
)
df_tags_named.head()

,goodreads_book_id,tag_id,count,tag_name
0,1,30574,167697,to-read
1,1,11305,37174,fantasy
2,1,11557,34173,favorites
3,1,8717,12986,currently-reading
4,1,33114,12716,young-adult


In [11]:
# List of generic tags to exclude (these describe user intent, not book content)
GENERIC_TAGS_TO_REMOVE = [
    'to-read',
    'currently-reading',
    'owned',
    'favorites',
    'default',
    'ebook',
    'kindle',
    'library',
    'owned-books',
    'my-books',
    'books'
]

# 1. Filter out generic tags
df_content_tags = df_tags_named[~df_tags_named['tag_name'].isin(GENERIC_TAGS_TO_REMOVE)].copy()

# Sort by count (descending) to prioritize the most important tags for each book
df_content_tags = df_content_tags.sort_values(
    ['goodreads_book_id', 'count'],
    ascending=[True, False]
)

# 2. Keep only the top 10 most relevant content tags per book (N=10)
N_TAGS = 10
df_top_tags_per_book = (
    df_content_tags.groupby('goodreads_book_id')
    .head(N_TAGS)
    .reset_index(drop=True)
)

# Aggregate the top 10 tag names into a single string ('tag_soup')
df_tags_grouped = (
    df_top_tags_per_book.groupby('goodreads_book_id')['tag_name']
    .apply(lambda x: ' '.join(x.astype(str).str.replace(' ', '-'))) # Join, replacing spaces with hyphens for better NLP
    .reset_index(name='tag_soup')
)

print("Aggregated Tags Sample:")
print(df_tags_grouped.head())

Aggregated Tags Sample:
   goodreads_book_id                                           tag_soup
0                  1  fantasy young-adult fiction harry-potter books...
1                  2  fantasy children children-s all-time-favorites...
2                  3  fantasy young-adult fiction harry-potter books...
3                  5  fantasy young-adult fiction harry-potter books...
4                  6  fantasy young-adult fiction harry-potter ya se...


Now merging this modified df_tags_grouped table with our main table

In [12]:




# 3. Final Merge with Main Books Table (using the goodreads_book_id)
df = pd.merge(
    df,
    df_tags_grouped,
    on='goodreads_book_id',
    how='left' 
)

df['tag_soup'] = df['tag_soup'].fillna('')

print("--- Final Books Content Data Sample ---")
print(df[['book_id', 'goodreads_book_id', 'title', 'authors', 'tag_soup']].head())

print(f"\nTotal rows in final dataframe: {len(df)}")

--- Final Books Content Data Sample ---
   book_id  goodreads_book_id  \
0        1            2767052   
1        2                  3   
2        3              41865   
3        4               2657   
4        5               4671   

                                               title  \
0            The Hunger Games (The Hunger Games, #1)   
1  Harry Potter and the Sorcerer's Stone (Harry P...   
2                            Twilight (Twilight, #1)   
3                              To Kill a Mockingbird   
4                                   The Great Gatsby   

                       authors  \
0              Suzanne Collins   
1  J.K. Rowling, Mary GrandPré   
2              Stephenie Meyer   
3                   Harper Lee   
4          F. Scott Fitzgerald   

                                            tag_soup  
0  young-adult fiction dystopian dystopia fantasy...  
1  fantasy young-adult fiction harry-potter books...  
2  young-adult fantasy vampires ya fiction parano...  

In [26]:
# Assuming df_books_content is your current DataFrame

# Function to clean and combine features
def clean_and_combine_features(row):
    # Ensure authors and title are clean and concatenated
    authors = str(row['authors']).replace(' ', '').replace(',', ' ')
    title = str(row['title']).replace(' ', '').replace(',', ' ')
    
    # The tag_soup already has spaces replaced by hyphens, which is good
    tags = str(row['tag_soup'])
    
    # Combine the key features into one string
    return authors + ' ' + title + ' ' + tags

# Create the new 'content_soup' column
df['content_soup'] = df.apply(clean_and_combine_features, axis=1)

# print("\nContent Soup Sample (Cleaned and Combined Features):")
# print(df[['title', 'content_soup']].head(2).to_string())
df.head()

,book_id,goodreads_book_id,authors,original_publication_year,original_title,title,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,tag_soup,content_soup
0,1,2767052,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",4.34,4780653,4942365,155254,young-adult fiction dystopian dystopia fantasy...,SuzanneCollins TheHungerGames(TheHungerGames #...
1,2,3,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,4.44,4602479,4800065,75867,fantasy young-adult fiction harry-potter books...,J.K.Rowling MaryGrandPré HarryPotterandtheSorc...
2,3,41865,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",3.57,3866839,3916824,95009,young-adult fantasy vampires ya fiction parano...,StephenieMeyer Twilight(Twilight #1) young-adu...
3,4,2657,Harper Lee,1960.0,To Kill a Mockingbird,To Kill a Mockingbird,4.25,3198671,3340896,72586,classics classic historical-fiction school clà...,HarperLee ToKillaMockingbird classics classic ...
4,5,4671,F. Scott Fitzgerald,1925.0,The Great Gatsby,The Great Gatsby,3.89,2683664,2773745,51992,classics fiction classic books-i-own literatur...,F.ScottFitzgerald TheGreatGatsby classics fict...


Preparing the ratings.csv data to be used for neural collaborative filtering

In [14]:
df_cf_input = pd.read_csv("ratings.csv")
# Assuming df_cf_input is your filtered ratings DataFrame

# 1a. Create dense ID mapping
user_to_index = {original_id: index for index, original_id in enumerate(df_cf_input['user_id'].unique())}
book_to_index = {original_id: index for index, original_id in enumerate(df_cf_input['book_id'].unique())}

df_cf_input['user_index'] = df_cf_input['user_id'].map(user_to_index)
df_cf_input['book_index'] = df_cf_input['book_id'].map(book_to_index)

n_users = len(user_to_index)
n_books = len(book_to_index)

# 1b. Create binary target (only using high ratings as positive)
POSITIVE_RATING_THRESHOLD = 4
df_positive = df_cf_input[df_cf_input['rating'] >= POSITIVE_RATING_THRESHOLD].copy()
df_positive['target'] = 1

print(f"Total Unique Users: {n_users}, Total Unique Books: {n_books}")
print(f"Number of Positive Interactions: {len(df_positive)}")



# --- Assumptions ---
# df_positive, n_users, n_books are defined
# NEGATIVE_SAMPLE_RATIO = 1 (or your desired ratio)

# 1. Get the set of all known interactions (CRITICAL for fast lookup)
all_interactions = set(zip(df_cf_input['user_index'], df_cf_input['book_index']))
positive_samples_count = len(df_positive)

# 2. Determine the target number of negative samples
NEGATIVE_SAMPLE_RATIO=1
target_neg_count = positive_samples_count * NEGATIVE_SAMPLE_RATIO

# --- OPTIMIZATION STARTS HERE ---

# 3. Generate a large pool of random (user, book) candidates
# We generate 1.5x the needed amount to account for overlaps with positive interactions
OVERSAMPLE_FACTOR = 1.5
total_candidates_to_generate = int(target_neg_count * OVERSAMPLE_FACTOR)

print(f"Generating {total_candidates_to_generate} candidate samples...")

# Vectorized generation of random user and book indices
random_user_indices = np.random.randint(0, n_users, total_candidates_to_generate)
random_book_indices = np.random.randint(0, n_books, total_candidates_to_generate)

# Create a temporary DataFrame of candidate negative samples
df_candidates = pd.DataFrame({
    'user_index': random_user_indices,
    'book_index': random_book_indices,
})

# 4. Filter out any samples that are already in the 'all_interactions' set

# Convert the candidates into tuples for fast set lookup
candidate_interactions = set(zip(df_candidates['user_index'], df_candidates['book_index']))

# Identify which candidates are actual interactions (collisions)
# This is a highly optimized set operation
valid_neg_interactions = list(candidate_interactions - all_interactions)

# 5. Select the required number of valid negative samples
# We must ensure we only take the exact number needed (target_neg_count)
df_valid_neg = pd.DataFrame(valid_neg_interactions, columns=['user_index', 'book_index'])

# Ensure we have enough and slice to the target count
if len(df_valid_neg) < target_neg_count:
    print("\nWARNING: Not enough unique negative samples generated. Increase OVERSAMPLE_FACTOR.")
    df_negative = df_valid_neg
else:
    df_negative = df_valid_neg.head(target_neg_count)

# Add the target column
df_negative['target'] = 0

# --- Final Consolidation ---

# 6. Combine positive and negative samples
df_training_final = pd.concat([df_positive[['user_index', 'book_index', 'target']], df_negative], ignore_index=True)

# 7. Shuffle the data
df_training_final = df_training_final.sample(frac=1).reset_index(drop=True)

# 8. Print Results
print(f"\nOptimization Complete.")
print(f"Total final training samples: {len(df_training_final)}")
print(f"Positive samples: {len(df_positive)}, Negative samples: {len(df_negative)}")
print("Final Training Data Sample:")
print(df_training_final.head())


Total Unique Users: 53424, Total Unique Books: 10000
Number of Positive Interactions: 4122111
Generating 6183166 candidate samples...


/tmp/ipykernel_207646/476226398.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_negative['target'] = 0



Optimization Complete.
Total final training samples: 8244222
Positive samples: 4122111, Negative samples: 4122111
Final Training Data Sample:
   user_index  book_index  target
0       32947        1595       0
1       52898        3294       0
2       43062         133       0
3       42834        3223       1
4        7919        9357       0


Now our nerual collaborative filtering step begins
Step 1 is to split our data into training and testing

In [15]:
from sklearn.model_selection import train_test_split

# Separate the features (inputs) from the target (output)
users = df_training_final['user_index'].values
books = df_training_final['book_index'].values
targets = df_training_final['target'].values

# Split into 80% training and 20% validation sets
X_train_u, X_val_u, X_train_i, X_val_i, y_train, y_val = train_test_split(
    users, books, targets, test_size=0.2, random_state=42, stratify=targets
)

print(f"\nTraining Samples: {len(X_train_u)}")
print(f"Validation Samples: {len(X_val_u)}")


Training Samples: 6595377
Validation Samples: 1648845


Step 2 is defining the base ncf model architecture

In [16]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Input, Flatten, Concatenate, Dense, Multiply
from tensorflow.keras.models import Model

# Define the model function again (ensure you have n_users and n_books in scope)
def build_ncf_model(num_users, num_items, embedding_dim=10):
    # --- Input Layers ---
    user_input = Input(shape=(1,), name='user_input')
    item_input = Input(shape=(1,), name='item_input')

    # --- Embedding Layers (Shared for GMF and MLP paths) ---
    # GMF Path Embeddings
    user_embedding_gmf = Embedding(input_dim=num_users, output_dim=embedding_dim, name='user_gmf_embedding')(user_input)
    item_embedding_gmf = Embedding(input_dim=num_items, output_dim=embedding_dim, name='item_gmf_embedding')(item_input)
    # MLP Path Embeddings
    user_embedding_mlp = Embedding(input_dim=num_users, output_dim=embedding_dim, name='user_mlp_embedding')(user_input)
    item_embedding_mlp = Embedding(input_dim=num_items, output_dim=embedding_dim, name='item_mlp_embedding')(item_input)

    # --- GMF Path (Linear Interaction) ---
    gmf_user_flat = Flatten()(user_embedding_gmf)
    gmf_item_flat = Flatten()(item_embedding_gmf)
    gmf_interaction = Multiply()([gmf_user_flat, gmf_item_flat])

    # --- MLP Path (Non-linear Interaction) ---
    mlp_user_flat = Flatten()(user_embedding_mlp)
    mlp_item_flat = Flatten()(item_embedding_mlp)
    mlp_interaction = Concatenate()([mlp_user_flat, mlp_item_flat])
    
    # Simple DNN layers for non-linear learning
    mlp_layer = Dense(embedding_dim * 2, activation='relu')(mlp_interaction)
    mlp_layer = Dense(embedding_dim, activation='relu')(mlp_layer)

    # --- Fusion Layer (Combine GMF and MLP) ---
    fusion = Concatenate()([gmf_interaction, mlp_layer])
    
    # --- Prediction Layer ---
    # Output uses sigmoid for binary classification (predicting P(interaction))
    output = Dense(1, activation='sigmoid', name='output')(fusion)

    model = Model(inputs=[user_input, item_input], outputs=output)
    return model

# Initialize and compile the model
EMBEDDING_DIM = 10 # Hyperparameter: determines the size of the latent factors
ncf_model = build_ncf_model(n_users, n_books, embedding_dim=EMBEDDING_DIM)
ncf_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
                  loss='binary_crossentropy') # Binary Cross-Entropy for binary classification (0 or 1)

print("\nNCF Model Architecture Summary:")
ncf_model.summary()

# Image of Neural Collaborative Filtering Architecture

2025-12-12 14:47:57.246436: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.



NCF Model Architecture Summary:


2025-12-12 14:48:02.753819: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ user_input          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ item_input          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ user_mlp_embedding  │ (None, 1, 10)     │    534,240 │ user_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ item_mlp_embedding  │ (None, 1, 10)     │    100,000 │ item_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_2 (Flatten) │ (None, 10)        │          0 │ user_mlp_embeddi… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_3 (Flatten) │ (None, 10)        │          0 │ item_mlp_embeddi… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ user_gmf_embedding  │ (None, 1, 10)     │    534,240 │ user_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ item_gmf_embedding  │ (None, 1, 10)     │    100,000 │ item_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 20)        │          0 │ flatten_2[0][0],  │
│ (Concatenate)       │                   │            │ flatten_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 10)        │          0 │ user_gmf_embeddi… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 10)        │          0 │ item_gmf_embeddi… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 20)        │        420 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply (Multiply) │ (None, 10)        │          0 │ flatten[0][0],    │
│                     │                   │            │ flatten_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 10)        │        210 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 20)        │          0 │ multiply[0][0],   │
│ (Concatenate)       │                   │            │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 1)         │         21 │ concatenate_1[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,269,131 (4.84 MB)

 Trainable params: 1,269,131 (4.84 MB)

 Non-trainable params: 0 (0.00 B)

Step 3 is training the model

In [17]:
# Train the model
BATCH_SIZE = 256
EPOCHS = 3 # Start with 10 epochs, adjust based on validation loss

history = ncf_model.fit(
    [X_train_u, X_train_i], y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    verbose=1,
    validation_data=([X_val_u, X_val_i], y_val)
)

print("\nModel Training Complete!")

Epoch 1/3
   39/25764 ━━━━━━━━━━━━━━━━━━━━ 10:51 25ms/step - loss: 0.6932

KeyboardInterrupt: 

now predicting

In [18]:
import numpy as np

def recommend_ncf_for_user(ncf_model, user_id, df_cf_input, book_to_index, n_books, top_k=10):
    
    # 1. Map the original user_id to the internal index
    user_index = user_to_index.get(user_id)
    
    if user_index is None:
        print(f"Error: User ID {user_id} not found in the trained dataset.")
        return []

    # 2. Identify books the user HAS interacted with (exclude these)
    user_interactions = df_cf_input[df_cf_input['user_id'] == user_id]['book_index'].unique()
    
    # 3. Identify all book indices the user HAS NOT interacted with (candidates)
    all_book_indices = np.arange(n_books)
    candidate_book_indices = np.setdiff1d(all_book_indices, user_interactions)
    
    # 4. Prepare input for the model
    # The model expects two arrays: (user_indices, book_indices)
    user_indices_array = np.full(len(candidate_book_indices), user_index)
    
    # 5. Predict scores (inference)
    predictions = ncf_model.predict([user_indices_array, candidate_book_indices], verbose=0).flatten()
    
    # 6. Rank the candidates
    # Get the indices that would sort the predictions in descending order
    top_k_indices = predictions.argsort()[-top_k:][::-1]
    
    # Get the corresponding book indices (the *internal* IDs)
    top_book_indices = candidate_book_indices[top_k_indices]
    top_scores = predictions[top_k_indices]
    
    # 7. Map back to original book_id (the key to your main books table)
    
    # Create the inverse map (index_to_book)
    index_to_book = {v: k for k, v in book_to_index.items()}
    
    # Map the internal index back to the external book_id
    top_book_ids = [index_to_book[idx] for idx in top_book_indices]

    # 8. Return results
    results = pd.DataFrame({
        'original_book_id': top_book_ids,
        'ncf_score': top_scores
    })
    
    return results

# Example Usage: Assuming you want recommendations for a reliable user with original user_id=2
# Replace 2 with a real user_id from your dataset
# user_id_to_test = 2 
# recommendations = recommend_ncf_for_user(ncf_model, user_id_to_test, df_cf_input, book_to_index, n_books)
# print(f"\nRecommendations for User {user_id_to_test}:\n", recommendations)

using the model to get for some user id, the top k books it would highly rate

In [19]:
# --- Find a Valid User ID to Test ---
# Get the first user_id from the set of users the model was trained on
user_id_to_test = df_cf_input['user_id'].iloc[0]

print(f"Generating NCF recommendations for User ID: {user_id_to_test}\n")

# Call your defined recommendation function
recommendations_ncf = recommend_ncf_for_user(
    ncf_model=ncf_model,
    user_id=user_id_to_test,
    df_cf_input=df_cf_input,
    book_to_index=book_to_index,
    n_books=n_books,
    top_k=10
)

print("--- Raw NCF Output (Original Book IDs and Scores) ---")
print(recommendations_ncf)

# Assuming your main book content table is named df
# Ensure you have loaded or kept df from earlier steps!

# Perform a merge to join the predicted scores with the book details
final_recommendations_ncf = pd.merge(
    recommendations_ncf,
    df[['book_id', 'title', 'authors', 'average_rating']], # Select key columns
    left_on='original_book_id',
    right_on='book_id',
    how='left'
)

# Clean up and display the final result
final_recommendations_ncf = final_recommendations_ncf.sort_values(
    'ncf_score', 
    ascending=False
).reset_index(drop=True)

# Rename for clarity
final_recommendations_ncf = final_recommendations_ncf.rename(
    columns={'ncf_score': 'Predicted Score (NCF)'}
)

print("\n--- Final Personalized Recommendations (NCF) ---")
print(final_recommendations_ncf[[
    'Predicted Score (NCF)', 
    'title', 
    'authors', 
    'average_rating'
]].head(10).to_string())

Generating NCF recommendations for User ID: 1

--- Raw NCF Output (Original Book IDs and Scores) ---
   original_book_id  ncf_score
0               158   0.512003
1                18   0.510697
2                 8   0.509005
3              4305   0.508754
4               513   0.508528
5               749   0.508216
6              4856   0.508060
7              3102   0.507970
8              6516   0.507941
9               117   0.507905

--- Final Personalized Recommendations (NCF) ---
   Predicted Score (NCF)                                                           title                                             authors  average_rating
0               0.512003          Charlie and the Chocolate Factory (Charlie Bucket, #1)                           Roald Dahl, Quentin Blake            4.10
1               0.510697     Harry Potter and the Prisoner of Azkaban (Harry Potter, #3)             J.K. Rowling, Mary GrandPré, Rufus Beck            4.53
2               0.509005             

# Content-Based Filtering (CBF) Model Implementation

The Content-Based Filtering component solves the critical **New User/New Item Cold Start Problem** by generating recommendations based purely on descriptive features of the books. This ensures that even users with no prior rating history receive relevant suggestions instantly, and new books (with no ratings) can still be recommended.

## 🎯 Core Mechanism: Item Similarity

Instead of relying on user behavior, the CBF model generates **book-to-book similarity scores** using their shared content.

### 1. Feature Consolidation (`tag_soup`)
Features like **title, authors, and descriptive tags** were aggregated into a single text block (`content_soup` or `tag_soup`) for each book.

### 2. Vectorization (TF-IDF)
The text features were converted into a **sparse numerical matrix** using the **Term Frequency-Inverse Document Frequency (TF-IDF)** scheme.  
- TF-IDF weights **rare, distinguishing tags** (e.g., 'Austen', 'dystopian') higher than **common terms** (e.g., 'book', 'fiction').

### 3. Similarity Matrix Generation
The final step involves calculating the **Cosine Similarity** between every book's TF-IDF vector:

\[
\text{Similarity}(B_i, B_j) = \text{CosineSimilarity}(\text{Vector}_{B_i}, \text{Vector}_{B_j})
\]

The result is a comprehensive **Item Similarity Matrix**, where we can instantly look up the content similarity score between any two books in the dataset.


In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Assume df is loaded and preprocessed with the 'tag_soup' column

print("Starting TF-IDF Vectorization...")

# 1. Initialize the TF-IDF Vectorizer
# max_df=0.9: Ignore terms that appear in more than 90% of documents (too common)
# min_df=5: Ignore terms that appear in fewer than 5 documents (too rare)
tfidf = TfidfVectorizer(stop_words='english', max_df=0.9, min_df=5)

# Fit the vectorizer to the 'tag_soup' and transform the text into a sparse matrix
tfidf_matrix = tfidf.fit_transform(df['tag_soup'])

print(f"TF-IDF Matrix Shape: {tfidf_matrix.shape}")
print("Calculating Cosine Similarity Matrix (This may take a moment)...")

# 2. Compute the Cosine Similarity Matrix
# The resulting matrix (N x N) holds the similarity score between every pair of books.
# This matrix will be the core of your CBF model.
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

print("Cosine Similarity Matrix Calculation Complete.")

Starting TF-IDF Vectorization...
TF-IDF Matrix Shape: (10000, 1164)
Calculating Cosine Similarity Matrix (This may take a moment)...
Cosine Similarity Matrix Calculation Complete.


In [21]:
# Create a mapping series to quickly lookup the internal index from the book_id
book_to_index_series = pd.Series(df.index, index=df['book_id']).drop_duplicates()

def recommend_cbf_by_book_id(book_id, cosine_sim_matrix, df_content, top_k=10):
    
    # 1. Get the internal index corresponding to the input book_id
    try:
        idx = book_to_index_series[book_id]
    except KeyError:
        return f"Error: Book ID {book_id} not found in content data."

    # 2. Get the similarity scores for this book against all other books
    # The scores are the row/column corresponding to 'idx' in the matrix
    sim_scores = list(enumerate(cosine_sim_matrix[idx]))

    # 3. Sort the books based on the similarity scores (descending)
    # We ignore the first element (index 0) because it's the book itself (similarity = 1.0)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_k + 1] 

    # 4. Get the book indices and similarity scores
    book_indices = [i[0] for i in sim_scores]
    scores = [i[1] for i in sim_scores]
    
    # 5. Get the original book IDs and titles for the top recommendations
    recommended_books = df_content.iloc[book_indices].copy()
    
    # 6. Format and return
    results = pd.DataFrame({
        'original_book_id': recommended_books['book_id'],
        'cbf_score': scores,
        'title': recommended_books['title'],
        'authors': recommended_books['authors']
    })
    
    return results.reset_index(drop=True)

# Example Usage: Find books similar to the book with original book_id=1
# You need to ensure book_id=1 is in your df
# recommendation_cbf = recommend_cbf_by_book_id(1, cosine_sim, df)
# print("\n--- Content-Based Recommendations for Book ID 1 ---")
# print(recommendation_cbf.to_string())

In [22]:
import pandas as pd
# Assuming df, cosine_sim, and the function recommend_cbf_by_book_id are defined

# --- 1. Select a Book ID to Use as the Anchor ---
# We will use the first book ID found in your content data as a reliable test case.
book_id_to_test = df['book_id'].iloc[70]

# --- 2. Retrieve the Title of the Anchor Book (For context in the output) ---
anchor_title = df[df['book_id'] == book_id_to_test]['title'].iloc[0]

print(f"--- Generating Content-Based Recommendations ---")
print(f"Anchor Book ID: {book_id_to_test}")
print(f"Anchor Book Title: {anchor_title}\n")

# --- 3. Run the Recommendation Function ---
recommendation_cbf = recommend_cbf_by_book_id(
    book_id=book_id_to_test,
    cosine_sim_matrix=cosine_sim,
    df_content=df,
    top_k=10 # Get the top 10 similar books
)

# --- 4. Display Results ---
print("Results based on shared tags/authors (Content-Based Score):")
print("----------------------------------------------------------")
# Use to_string() for clean console display
print(recommendation_cbf[['cbf_score', 'title', 'authors']].to_string())

--- Generating Content-Based Recommendations ---
Anchor Book ID: 71
Anchor Book Title: Frankenstein

Results based on shared tags/authors (Content-Based Score):
----------------------------------------------------------
   cbf_score                                                                  title                                                            authors
0   0.710223                                             The Picture of Dorian Gray                                     Oscar Wilde, Jeffrey Eugenides
1   0.698998                            The Strange Case of Dr. Jekyll and Mr. Hyde  Robert Louis Stevenson, Vladimir Nabokov, Mervyn Peake, Dan Chaon
2   0.624072                                                                Dracula                          Bram Stoker, Nina Auerbach, David J. Skal
3   0.622619                                     Necronomicon: The Best Weird Tales                         H.P. Lovecraft, Les Edwards, Stephen Jones
4   0.606474             

The key challenge here is that the NCF function gives scores for all unseen items, while the CBF function only gives item-to-item similarity scores based on a single anchor book.

For a true hybrid, we need to adapt the CBF logic to a User-Centric CBF Score:

    Identify User's Favorite Books: Find the books the user rated highest.

    Calculate Average Similarity: For every candidate book, calculate its average Content-Based Similarity to the user's favorite books. This gives us a Scbf​ score for that user/item pair.

    Fuse Scores: Combine Sncf​ and Scbf​.

1. Adaptation: User-Centric CBF Score (Required Pre-Calculation)

First, we need a function that calculates the user-centric CBF score for all candidate books.

In [23]:
def calculate_user_cbf_scores(user_id, df_cf_input, df_positive, book_to_index_series, cosine_sim_matrix, candidate_book_indices):
    
    # 1. Identify the user's high-rated books (using internal row indices)
    # We use df_positive (filtered high ratings) for a stronger signal
    user_pos_ratings = df_positive[df_positive['user_id'] == user_id]
    
    # Get the row indices from df_books_content for these positive books
    # Note: We are using the original book_id here to map to the content DataFrame index
    user_fav_book_ids = user_pos_ratings['book_id'].unique()
    
    # Filter out any favorites that might not be in the content DataFrame
    valid_fav_indices = [book_to_index_series[bid] for bid in user_fav_book_ids if bid in book_to_index_series]
    
    if not valid_fav_indices:
        return pd.Series(0.0, index=candidate_book_indices) # Return 0 scores if user has no favorites

    # 2. Calculate the average content similarity for all candidates
    
    # Extract the similarity scores for all candidate books across the rows of the user's favorites
    # This gives us a (Num_Favorites x Num_Candidates) matrix
    sims_to_favorites = cosine_sim_matrix[valid_fav_indices][:, candidate_book_indices]
    
    # Average the scores across the favorites axis (axis=0) to get one score per candidate book
    avg_cbf_scores = np.mean(sims_to_favorites, axis=0)
    
    # Return as a pandas Series for easy merging later (indexed by candidate book index)
    return pd.Series(avg_cbf_scores, index=candidate_book_indices)

2. The Final Hybrid Function

This function puts the NCF score, the new CBF score, and the weighted fusion together.

In [24]:
def recommend_hybrid(user_id, ncf_model, df_cf_input, df_positive, book_to_index, book_to_index_series, n_books, cosine_sim_matrix, top_k=10, w_ncf=0.7, w_cbf=0.3):
    
    # --- Part A: Get NCF Scores and Candidate List ---
    
    # 1. Run the NCF prediction function (we adapt the logic slightly for efficiency)
    # Re-use the logic from recommend_ncf_for_user to get the base predictions and candidates
    
    # Get user index
    user_index = user_to_index.get(user_id)
    if user_index is None:
        return "User not found."

    # Identify candidates
    user_interactions = df_cf_input[df_cf_input['user_id'] == user_id]['book_index'].unique()
    all_book_indices = np.arange(n_books)
    candidate_book_indices = np.setdiff1d(all_book_indices, user_interactions)
    
    # Run NCF prediction
    user_indices_array = np.full(len(candidate_book_indices), user_index)
    ncf_scores_array = ncf_model.predict([user_indices_array, candidate_book_indices], verbose=0).flatten()
    
    # Create base DataFrame
    df_scores = pd.DataFrame({
        'book_index': candidate_book_indices,
        'ncf_score': ncf_scores_array
    }).set_index('book_index')

    
    # --- Part B: Get CBF Scores ---

    # 2. Calculate the User-Centric CBF Score for all candidates
    cbf_scores_series = calculate_user_cbf_scores(
        user_id, df_cf_input, df_positive, book_to_index_series, cosine_sim_matrix, candidate_book_indices
    )
    
    # Merge CBF scores onto the main score DataFrame
    df_scores['cbf_score'] = cbf_scores_series
    
    
    # --- Part C: Fusion and Final Ranking ---
    
    # 3. Apply the Hybrid Fusion Formula
    df_scores['hybrid_score'] = (w_ncf * df_scores['ncf_score']) + (w_cbf * df_scores['cbf_score'])
    
    # 4. Final Ranking
    top_recommendations = df_scores.sort_values('hybrid_score', ascending=False).head(top_k)
    
    
    # --- Part D: Mapping and Formatting ---
    
    # Create the inverse map (index_to_book)
    index_to_book = {v: k for k, v in book_to_index.items()}
    
    # Map the internal index back to the external book_id
    top_recommendations['original_book_id'] = [index_to_book[idx] for idx in top_recommendations.index]
    
    return top_recommendations.reset_index().drop(columns=['book_index'])


# --- Example Usage (Requires all variables to be defined) ---

# Find a valid user to test (e.g., the first user in the positive ratings)
# user_id_to_test = df_positive['user_id'].iloc[0]

# final_recs = recommend_hybrid(
#     user_id=user_id_to_test,
#     ncf_model=ncf_model,
#     df_cf_input=df_cf_input,
#     df_positive=df_positive,
#     book_to_index=book_to_index,
#     book_to_index_series=book_to_index_series,
#     n_books=n_books,
#     cosine_sim_matrix=cosine_sim,
#     w_ncf=0.7, # 70% weight to personalization
#     w_cbf=0.3  # 30% weight to relevance/content
# )

# print(f"\n--- Final Hybrid Recommendations for User {user_id_to_test} ---")
# print(final_recs)

# To display titles, you would merge final_recs with df_books_content on 'original_book_id'

In [27]:

# Assumes: ncf_model, df_cf_input, df_positive, book_to_index, book_to_index_series, 
# n_books, cosine_sim_matrix, and df_books_content are all defined.
# Assumes: The recommend_hybrid and calculate_user_cbf_scores functions are defined.

# --- 1. Find a Valid User ID to Test ---
# We select a user who has positive ratings, ensuring both NCF and CBF signals are active.
user_id_to_test = df_positive['user_id'].iloc[0] 

# --- 2. Define Fusion Weights ---
# These are the hyperparameters that control the balance of the hybrid system.
W_NCF = 0.7  # Weight for personalized taste (NCF)
W_CBF = 0.3  # Weight for content relevance (CBF/TF-IDF)

print(f"--- Generating HYBRID Recommendations for User ID: {user_id_to_test} ---")
print(f"Fusion Weights: NCF={W_NCF}, CBF={W_CBF}\n")

# --- 3. Execute the Hybrid Recommendation Function ---
final_recs_raw = recommend_hybrid(
    user_id=user_id_to_test,
    ncf_model=ncf_model,
    df_cf_input=df_cf_input,
    df_positive=df_positive,
    book_to_index=book_to_index,
    book_to_index_series=book_to_index_series,
    n_books=n_books,
    cosine_sim_matrix=cosine_sim,
    top_k=10, 
    w_ncf=W_NCF, 
    w_cbf=W_CBF
)

# --- 4. Merge Raw Scores with Book Content for Readability ---
# Join the raw results (which contain the calculated scores) with the main content DataFrame
final_recs_display = pd.merge(
    final_recs_raw,
    df[['book_id', 'title', 'authors', 'average_rating']],
    left_on='original_book_id',
    right_on='book_id',
    how='left'
)

# --- 5. Display Final Results ---

print("Final Hybrid Score Breakdown (Top 10):")
print("--------------------------------------")

# Select and rename columns for a clean output table
output_columns = {
    'hybrid_score': 'Hybrid Score',
    'ncf_score': 'NCF (Taste)',
    'cbf_score': 'CBF (Content)',
    'title': 'Book Title',
    'authors': 'Authors',
    'average_rating': 'Avg Rating'
}

final_recs_display = final_recs_display.rename(columns=output_columns)

# Display the final, beautifully organized table
print(final_recs_display[[
    'Hybrid Score',
    'NCF (Taste)',
    'CBF (Content)',
    'Book Title',
    'Authors',
    'Avg Rating'
]].to_string(index=False))

--- Generating HYBRID Recommendations for User ID: 1 ---
Fusion Weights: NCF=0.7, CBF=0.3

Final Hybrid Score Breakdown (Top 10):
--------------------------------------
 Hybrid Score  NCF (Taste)  CBF (Content)                                                                           Book Title                                            Authors  Avg Rating
     0.445813     0.502352       0.313888                                                                                Proof                                       David Auburn        4.01
     0.441307     0.500921       0.302209                The Millionaire Next Door: The Surprising Secrets of Americas Wealthy                Thomas J. Stanley, William D. Danko        4.00
     0.440193     0.503128       0.293345                                                                          The Prophet                                      Kahlil Gibran        4.22
     0.439903     0.501527       0.296115                              